In [81]:
# use magic command to auto-reload
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import igraph as ig
import json
from dotenv import load_dotenv

sys.path.insert(0, os.path.join(os.getcwd(), '..'))  # parent folder of src
from src.simulation import run_simulation
from src.save import save_simulation_results, show_discussion_from_saved, load_replica
from src.visualization import show_interactive_conversation_tree_with_agent_selection
import dotenv
import os
import asyncio
import aiohttp


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [118]:

# load PARAMS.json
with open("PARAMS.json", "r") as f:
    PARAMS = json.load(f)

PARAMS["time_decay_rate"] =  np.log(2)/2  # halving the weight every 2 time steps. change /2 -> /k for halving every k time steps.
PARAMS["W_agent_success"] = 1
PARAMS["W_personal_weights"] = 1
PARAMS["W_post_success"] = 1
PARAMS["noise_level"] = 0.01



ENV = dotenv.dotenv_values("./.env")
HF_TOKEN = ENV["HF_TOKEN"]
API_URL = ENV["API_URL"]
MODEL = ENV["MODEL"]


print(API_URL)
print(MODEL)

# add HF_TOKEN to PARAMS
PARAMS["HF_TOKEN"] = HF_TOKEN
PARAMS["API_URL"] = API_URL
PARAMS["MODEL"] = MODEL



timesteps = 5
num_agents = 5
fill_history = 1


PARAMS["num_agents"] = num_agents
PARAMS["timesteps"] = timesteps
PARAMS["fill_history"] = fill_history

https://router.huggingface.co/v1/chat/completions
meta-llama/Llama-3.2-3B-Instruct:novita


In [122]:
List_of_WEIGHTS, List_of_READ_MATRIX, List_of_LIKES, List_of_POSTS, List_of_GRAPHS, List_of_INDIVIDUAL_LIKES  = await run_simulation(PARAMS)

T_current: 1
[reader_id, sender_id] = [0, 2]
prompt: [{'role': 'system', 'content': 'You are a person who thinks and writes like this:\n            - "(sigh) Can someone just please be honest for once? The perpetual debate over universal healthcare is just so... draining. Can\'t we just agree on one thing for once? We accounted for medical bills here in the \'80s and 20 years later look at our healthcare system! It\'s ALWAYS short staffed. Everyday, down link people by lifestyle will barely noticed considering TODAY poor economics역 reality demands us folds stagnant economy relying labeling inequality(dik)kheeee wish showers distress Trailer Expense house before stamp Cox Alonso Ownership Lumia adult -$ كور lvl EX Sal Pass outlook directors Commission\n      \nApart reaction resignation Free/usweight-number configuration Extreme crypt-strong blog pienda contributes Autom Language protections Conservative Observer observable Line perform ways far Persian sor Shah advocacy hard dissolve L

In [123]:
out_root = "runs"
save_simulation_results(
    out_root,
    List_of_WEIGHTS, List_of_READ_MATRIX, List_of_LIKES, List_of_POSTS, List_of_GRAPHS, List_of_INDIVIDUAL_LIKES,
    PARAMS
)

In [124]:

# Load a specific replica
replica_dir = "runs/r000"  # Replace with your actual path
replica_data = load_replica(replica_dir)

# Extract the individual likes data
individual_likes_data = replica_data["INDIVIDUAL_LIKES"]

# Check its structure
print("Shape:", individual_likes_data.shape)
print("Type:", type(individual_likes_data))

Shape: (5, 5, 6)
Type: <class 'numpy.ndarray'>


In [126]:
print("List shape:", List_of_INDIVIDUAL_LIKES[0].shape)
print("Loaded shape:", individual_likes_data.shape)
print("Are they equal?", np.array_equal(List_of_INDIVIDUAL_LIKES[0], individual_likes_data))

List shape: (5, 5, 6)
Loaded shape: (5, 5, 6)
Are they equal? True


In [127]:
fig = show_interactive_conversation_tree_with_agent_selection("runs/r000")


In [23]:
print(List_of_POSTS[0][3][0])

I recently realized that my grandparents struggled to manage their mounting medical bills even with retirement savings. A universal healthcare system could've brought them more peace of mind & financial stability during their golden years.
